# Video analysis using YOLOv5

In this tutorial, you will apply the Object Detection model (YOLOv5) to a video.

---
### Mount to Google Drive
Mount to your Google Drive to access the images and model files.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DataFolder = "/content/drive/MyDrive/SDAAI/data"

---
## Install the dependencies

In [ ]:
# Install the necessary dependencies
!pip install opencv-python
!pip install glib
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

---
## Import the necessary libraries

In [ ]:
import cv2
%matplotlib inline
from matplotlib import pyplot as plt
from tqdm import tqdm

---
## Create the YOLOv5 model

In [ ]:
# Import the neccesary libraries
import torch

# Load the Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  

---
## Define ```annotate_image()```

Define a function to take in am RGB image, perform object detection on the image and return the annotated image.

In [ ]:
def annotate_image(image):
    
    # perform precidtion
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model(image_rgb) 
    
    for i in range(len(results.pandas().xyxy[0].name)):
      name = results.pandas().xyxy[0].name[i]
      startX = int(results.pandas().xyxy[0].xmin[i])
      startY = int(results.pandas().xyxy[0].ymin[i])
      endX = int(results.pandas().xyxy[0].xmax[i])
      endY = int(results.pandas().xyxy[0].ymax[i])
      confidence = results.pandas().xyxy[0].confidence[i]
      label = "{}: {:.2f}%".format(name, confidence * 100)
      if confidence > 0.6:
        cv2.rectangle(image, (startX, startY), (endX, endY),
                  (255,0,0), 2)
        y = startY - 15 if startY - 15 > 15 else startY + 15
        cv2.putText(image, label, (startX, y),
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2)
    
    return image    

---
## Test the ```annotate_image()```

In [ ]:
image = cv2.imread(DataFolder + '/LU04_images/zebra.jpg')
ann_image = annotate_image(image)

image_display = cv2.cvtColor(ann_image, cv2.COLOR_BGR2RGB)
plt.imshow(image_display)
plt.show()

---
Process the video file

In [ ]:
video_in_file = DataFolder + "/LU06_videos/Demo1.mp4"
video_out_file = DataFolder + "/LU06_videos/out.mp4"

print("[INFO] accessing video stream...")
v_out = None

v_in = cv2.VideoCapture(video_in_file)
total_frame = int(v_in.get(cv2.CAP_PROP_FRAME_COUNT ))

for frame_no in tqdm(range(total_frame), desc="Processing Video..."):

  (grabbed, frame) = v_in.read()

  # if the frame was not grabbed then we've reached the end of
  # the video stream so exit the script
  if not grabbed:
      print("[INFO] no frame read from stream - exiting")
      break
          
  annotated_img = annotate_image(frame)
      
  # check if the video writer is None
  if v_out is None:
      # initialize our video writer
      fourcc = cv2.VideoWriter_fourcc(*"mp4v")
      v_out = cv2.VideoWriter(video_out_file, fourcc, 
                  int(v_in.get(cv2.CAP_PROP_FPS)),
                  (frame.shape[1], frame.shape[0]), True) 

  # write the output frame to disk
  v_out.write(annotated_img)
    
# release the file pointers
print("\n[INFO] cleaning up...")
v_out.release()
v_in.release()

---
Play the out.mp4 file.

In [ ]:
video_mp4 = DataFolder + "/LU06_videos/out_2.mp4"
!ffmpeg -y -loglevel info -i $video_out_file -vf scale=640:480 $video_mp4

In [ ]:
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

In [ ]:
show_local_mp4_video(video_mp4, width=640, height=480)